In [ ]:
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

## Download Data

In [ ]:
train, test = tf.keras.datasets.mnist.load_data()

In [ ]:
type(train), len(train)

In [ ]:
train_x, train_y = train
test_x, test_y = test

In [ ]:
train_x.shape, train_y.shape

In [ ]:
train_x = np.asarray(train_x, dtype=float)
train_y = np.asarray(train_y, dtype=int)
test_x = np.asarray(test_x, dtype=float)
test_y = np.asarray(test_y, dtype=int)

### Sample data

In [ ]:
nx, ny = 5, 5
fig, ax = plt.subplots(ny, nx)
fig.subplots_adjust(wspace=-0.7, hspace=0.1)
for i in range(ny):
    for j in range(nx):
        ax[i][j].imshow(train_x[ny*i+j], cmap='gray')
        ax[i][j].axis('off')

plt.show()        
print(np.reshape(train_y[:nx * ny], [nx, ny]))

# Neural Network

![Neural Network](nn.gif)

### Dense Neural Network

In [ ]:
# Build the graph for dense neural network.
def build_dnn(input_size, hidden_layers, output_size):
    x = tf.placeholder(tf.float32, shape=[None, input_size])
    y_ = tf.placeholder(tf.int64, shape=(None,))
    prev_size = input_size
    
    h = x
    for i, layer_size in enumerate(hidden_layers):
        ### FIX ME
    
    with tf.name_scope("fc_out"):
        ### FIX ME
        
    return x, logits, y_

In [ ]:
# Data stream.
def data_batches(features, labels, batch_size, shuffle=False):
    if shuffle:
        perm = np.random.permutation(len(features))
        features = features[perm]
        labels = labels[perm]
        
    i = 0
    while i < len(features):
        yield features[i:i+batch_size], labels[i:i+batch_size]
        i += batch_size

In [ ]:
tf.reset_default_graph()

x, logits, y_ = build_dnn(28*28, [2000, 2000], 10)

loss = 
train_op = 

prediction = 
accuracy = 

In [ ]:
# Start a Session.
sess = 
# Create a writer.
writer = 

In [ ]:
# Training
start_time = time.time()
for step in range(2000):
    # if (step + 1) % 100 == 0 or step == 0:
    #     test_accuracy = 
    #     train_loss = 
        
        # Add scalar summary to tensorboard
        # writer.add_summary(tf.Summary(value=[tf.Summary.Value(tag="test_accuracy", simple_value=test_accuracy),
        #                                      tf.Summary.Value(tag="train_loss", simple_value=train_loss)]),
        #                    step)
        
        # Print basic training info.
    #     dtime = time.time() - start_time
    #     start_time = time.time()
    #     print("Step {} ({:.3f}s), loss: {}, test accuracy: {} \t ".format(
    #             step+1, dtime, train_loss, test_accuracy))
    
    
    train_data_batches = data_batches(train_x, train_y, batch_size=None, shuffle=True)
    for batch_x, batch_y in train_data_batches:
        ### FIX ME
        
writer.close()

In [ ]:
nx, ny = 5, 5
fig, ax = plt.subplots(ny, nx)
fig.subplots_adjust(wspace=-0.7, hspace=0.1)
for i in range(ny):
    for j in range(nx):
        ax[i][j].imshow(test_x[ny*i+j], cmap='gray')
        ax[i][j].axis('off')

plt.show()   

predicted = sess.run(prediction, feed_dict={x: np.reshape(test_x[:nx*ny], [-1, 28*28])})
print(np.reshape(predicted, [nx, ny]))

In [ ]:
sess.close()

# Convolution Neural Network

[Convolution Animation](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md)

In [ ]:
def build_cnn(input_size, channels, fc_layers, output_size):
    x = tf.placeholder(tf.float32, shape=[None, input_size, input_size])
    y_ = tf.placeholder(tf.int64, shape=[None])
    
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, input_size, input_size, 1])
    
    prev = x_image
    prev_c = 1
    for i, c in enumerate(channels):
        with tf.name_scope('conv{}'.format(i)):
            W_conv = tf.Variable(tf.truncated_normal([3, 3, prev_c, c]))
            h_conv = tf.nn.relu(tf.nn.conv2d(prev, W_conv, strides=[1, 1, 1, 1], padding='SAME'))
        
        with tf.name_scope('pool{}'.format(i)):
            h_pool = tf.nn.max_pool(h_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            
        prev = h_pool
        prev_c = c
    
    h = tf.contrib.layers.flatten(h_pool)
    prev_size = h.get_shape().as_list()[1]
    for i, layer_size in enumerate(fc_layers):
        with tf.name_scope("fc{}".format(i)):
            W = tf.Variable(tf.truncated_normal([prev_size, layer_size]), name="W")
            b = tf.Variable(tf.zeros([layer_size]), name="b")
            
            h = tf.nn.relu(tf.matmul(h, W) + b)
            
        prev_size = layer_size
        
    with tf.name_scope("fc_out"):
        W = tf.Variable(tf.truncated_normal([prev_size, output_size]), name="W")
        b = tf.Variable(tf.zeros([output_size]), name="b")
        
        logits = tf.matmul(h, W) + b
        
    return x, logits, y_

In [ ]:
tf.reset_default_graph()

x, logits, y_ = build_cnn(28, [32, 64], [1000], 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=logits)
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)

prediction = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_), tf.float32))

In [ ]:
sess = tf.Session()
writer = tf.summary.FileWriter("mnist_cnn_raw")

writer.add_graph(sess.graph)

sess.run(tf.global_variables_initializer())

start_time = time.time()
for step in range(300):
    if (step + 1) % 100 == 0 or step == 0:
        test_accuracy = sess.run(accuracy, feed_dict={x: test_x, y_: test_y})
        sample = np.random.choice(len(train_x), 1000, replace=False)
        train_loss = sess.run(loss, feed_dict={x: train_x[sample], y_: train_y[sample]})
        
        writer.add_summary(tf.Summary(value=[tf.Summary.Value(tag="test_accuracy", simple_value=test_accuracy),
                                             tf.Summary.Value(tag="train_loss", simple_value=train_loss)]),
                           step)
        
        dtime = time.time() - start_time
        start_time = time.time()
        print("Step {} ({:.3f}s), loss: {}, test accuracy: {} \t ".format(
                step+1, dtime, train_loss, test_accuracy))
        
    train_data_batches = data_batches(train_x, train_y, batch_size=128, shuffle=True)
    for batch_x, batch_y in train_data_batches:
        sess.run(train_op, feed_dict={x: batch_x, y_: batch_y})
        
writer.close()

In [ ]:
nx, ny = 5, 5
fig, ax = plt.subplots(ny, nx)
fig.subplots_adjust(wspace=-0.7, hspace=0.1)
for i in range(ny):
    for j in range(nx):
        ax[i][j].imshow(test_x[ny*i+j], cmap='gray')
        ax[i][j].axis('off')

plt.show()   

predicted = sess.run(prediction, feed_dict={x: np.reshape(test_x[:nx*ny], [-1, 28*28])})
print(np.reshape(predicted, [nx, ny]))

In [ ]:
sess.close()

### Dense Neural Network with `Estimator`

In [ ]:
feature_columns = [tf.feature_column.numeric_column(key='x', shape=(28, 28))]

In [ ]:
mnist_dnn_classifier = tf.estimator.DNNClassifier(
                        feature_columns=feature_columns,
                        hidden_units=[2000, 200],
                        n_classes=10,
                        model_dir="mnist_dnn_estimator")

In [ ]:
def train_input_fn(features:dict, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [ ]:
def eval_input_fn(features:dict, labels, batch_size):
    if labels is None:
        return tf.data.Dataset.from_tensor_slices(features).batch(batch_size)
    
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(batch_size)

In [ ]:
mnist_dnn_classifier.train(input_fn=lambda: train_input_fn({"x": train_x}, train_y, 64), steps=500)

### CNN model with custom `Estimator`

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
                inputs=input_layer,
                filters=32,
                kernel_size=[5, 5],
                padding="same",
                activation=tf.nn.relu)
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
                inputs=pool1,
                filters=64,
                kernel_size=[5, 5],
                padding="same",
                activation=tf.nn.relu)
    
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
                inputs=dense, rate=0.4, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT  and by the `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # `sparse_softmax_cross_entropy` takes class number as label
    # `softmax_cross_entropy` takes onehot vector as label
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
                    loss=loss,
                    global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
                        labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
                mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
mnist_classifier = tf.estimator.Estimator(
                        model_fn=cnn_model_fn, model_dir="mnist_convnet_model")

In [ ]:
mnist_classifier.train(
    input_fn=lambda: train_input_fn({"x": train_x}, train_y, batch_size=64),
    steps=2000)

In [ ]:
eval_results = mnist_classifier.evaluate(input_fn=lambda: eval_input_fn({"x": test_x}, test_y, batch_size=64))

In [ ]:
print(eval_results)